<a href="https://colab.research.google.com/github/DaraRahma536/ML/blob/main/UTS/fraud_1/UTS1_ML_Dara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xgboost==2.0.3 polars
!pip install optuna
!pip install polars

In [1]:
import pandas as pd
import numpy as np
import polars as pl

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, classification_report
from lightgbm import LGBMClassifier
from sklearn.impute import SimpleImputer

In [2]:
!pip install -q gdown

# kalian download semua file dari gdrive dibawah lalu buat folder (ex. midterm_folder)
!gdown --folder 1qoyfQ0yXp9pgnp9zKwsRm0B-u0n-KopN -O midterm_folder

Retrieving folder contents
Processing file 1FlTQDZijxi_AdhdPlxvcgKdSRlcbVrKk test_transaction.csv
Processing file 18BxWO8J8QgLfWeFInaLnswSph4yNg2RL train_transaction.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1FlTQDZijxi_AdhdPlxvcgKdSRlcbVrKk
From (redirected): https://drive.google.com/uc?id=1FlTQDZijxi_AdhdPlxvcgKdSRlcbVrKk&confirm=t&uuid=d51aeec0-e671-482c-833b-8edfc7fe4e57
To: /content/midterm_folder/test_transaction.csv
100% 613M/613M [00:07<00:00, 79.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=18BxWO8J8QgLfWeFInaLnswSph4yNg2RL
From (redirected): https://drive.google.com/uc?id=18BxWO8J8QgLfWeFInaLnswSph4yNg2RL&confirm=t&uuid=78cdcaeb-36f2-4afa-a6c1-cafea7f2eea9
To: /content/midterm_folder/train_transaction.csv
100% 683M/683M [00:05<00:00, 126MB/s]
Download completed


In [3]:
BASE_PATH = "midterm_folder"

train_transaction = pl.read_csv(f"{BASE_PATH}/train_transaction.csv")
test_transaction  = pl.read_csv(f"{BASE_PATH}/test_transaction.csv")

# print(train_transaction.shape) #590540 row dan 393 feature + 1 target
# print(test_transaction.shape) #506691 row dan 393 feature

In [ ]:
train_transaction.head() #isFraud adalah target featurenya

TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
i64,i64,i64,f64,str,i64,f64,f64,str,f64,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2987000,0,86400,68.5,"""W""",13926,null,150.0,"""discover""",142.0,"""credit""",315.0,87.0,19.0,null,null,null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,null,13.0,null,null,null,…,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987001,0,86401,29.0,"""W""",2755,404.0,150.0,"""mastercard""",102.0,"""credit""",325.0,87.0,null,null,"""gmail.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987002,0,86469,59.0,"""W""",4663,490.0,150.0,"""visa""",166.0,"""debit""",330.0,87.0,287.0,null,"""outlook.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987003,0,86499,50.0,"""W""",18132,567.0,150.0,"""mastercard""",117.0,"""debit""",476.0,87.0,null,null,"""yahoo.com""",null,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,null,…,0.0,0.0,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987004,0,86506,50.0,"""H""",4497,514.0,150.0,"""mastercard""",102.0,"""credit""",420.0,87.0,null,null,"""gmail.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,null,null,null,null,null,…,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test_transaction.head() #uji data baru, setelah pembuatan model machine learningnya

TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
i64,i64,f64,str,i64,f64,f64,str,f64,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3663549,18403224,31.95,"""W""",10409,111.0,150.0,"""visa""",226.0,"""debit""",170.0,87.0,1.0,null,"""gmail.com""",null,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,null,null,…,0.0,0.0,1.0,0.0,47.950001,0.0,0.0,47.950001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663550,18403263,49.0,"""W""",4272,111.0,150.0,"""visa""",226.0,"""debit""",299.0,87.0,4.0,null,"""aol.com""",null,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,null,null,…,0.0,0.0,1.0,0.0,280.0,77.0,0.0,280.0,0.0,77.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663551,18403310,171.0,"""W""",4476,574.0,150.0,"""visa""",226.0,"""debit""",472.0,87.0,2635.0,null,"""hotmail.com""",null,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,null,null,…,0.0,0.0,1.0,0.0,1321.0,0.0,0.0,1058.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663552,18403310,284.95,"""W""",10989,360.0,150.0,"""visa""",166.0,"""debit""",205.0,87.0,17.0,null,"""gmail.com""",null,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,282.540009,282.540009,282.540009,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663553,18403317,67.95,"""W""",18018,452.0,150.0,"""mastercard""",117.0,"""debit""",264.0,87.0,6.0,null,"""gmail.com""",null,6.0,6.0,0.0,0.0,2.0,5.0,0.0,0.0,5.0,0.0,6.0,0.0,14.0,6.0,22.0,22.0,0.0,22.0,0.0,null,null,…,0.0,0.0,1.0,67.949997,183.850006,67.949997,67.949997,183.850006,67.949997,67.949997,67.949997,183.850006,67.949997,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


### **Pakai CPU Saja**

### **4.0**

In [13]:
!pip install optuna-integration[lightgbm]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 2.4 MB/s eta 0:00:00


In [21]:
!pip install --upgrade lightgbm optuna

In [26]:
# ======================================================
# 1. IMPORT LIBRARIES
# ======================================================
import polars as pl
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import optuna
import os
import optuna.integration.lightgbm as lgb_tuner

# from optuna.integration import lightgbm as lgb_tuner
# from lightgbm import LightGBMTunerCV
from sklearn.model_selection import StratifiedKFold


In [5]:
# ======================================================
# 2. LOAD DATASET WITH POLARS
# ======================================================
# train_transactions = pl.read_csv("train_transaction.csv")
# test_transactions  = pl.read_csv("test_transaction.csv")

target = "isFraud"

In [6]:
# =======================================================
# 3. SPLIT NUMERIC & CATEGORICAL COLUMNS & IMPUTATION
# =======================================================
num_cols = [c for c, t in zip(train_transaction.columns, train_transaction.dtypes)
            if t in (pl.Float32, pl.Float64, pl.Int64, pl.Int32) and c != target]

cat_cols = [c for c, t in zip(train_transaction.columns, train_transaction.dtypes)
            if t == pl.Utf8]

# numeric → median
for col in num_cols:
    med = train_transaction[col].median()
    train_transaction = train_transaction.with_columns(pl.col(col).fill_null(med))
    test_transaction  = test_transaction.with_columns(pl.col(col).fill_null(med))

# categorical → missing
for col in cat_cols:
    train_transaction = train_transaction.with_columns(pl.col(col).fill_null("missing"))
    test_transaction  = test_transaction.with_columns(pl.col(col).fill_null("missing"))

In [9]:
# =======================================================
# 4. LABEL ENCODING (HANYA POLARS)
# =======================================================
for col in cat_cols:
    # Ambil seluruh nilai unik
    unique_vals = (
        pl.concat([train_transaction[col], test_transaction[col]])
        .unique()
        .to_list()
    )

    # Buat mapping integer
    mapping = {v: i for i, v in enumerate(unique_vals)}

    # Terapkan ke train & test (Polars)
    train_transaction = train_transaction.with_columns(
        pl.col(col).replace(mapping).cast(pl.Int64)
    )
    test_transaction = test_transaction.with_columns(
        pl.col(col).replace(mapping).cast(pl.Int64)
    )

In [19]:
# ======================================================
# 5. SAVE CLEANED DATA TO CSV (FOR STREAMING MODE)
# ======================================================

# train_transaction.write_csv("train_clean.csv")
# test_transaction.write_csv("test_clean.csv")

# # Save labels separately (LightGBM needs it)
# y_list = train_transaction[target].to_list()
# pl.DataFrame({"isFraud": y_list}).write_csv("train_label.csv")

# feature_cols = [c for c in train_transaction.columns if c != target]


# train_transaction = train_transaction.sample(frac=1.0, shuffle=True)

# val_size = int(0.2 * train_transaction.height)

# val_df = train_transaction[:val_size]
# train_transaction2 = train_transaction[val_size:]

# X_train = train_transaction2.drop(target).to_numpy()
# y_train = train_transaction2[target].to_numpy()

# X_val   = val_df.drop(target).to_numpy()
# y_val   = val_df[target].to_numpy()



# train_lgb = lgb.Dataset(X_train, label=y_train)
# valid_lgb = lgb.Dataset(X_val, label=y_val)

In [9]:
# ======================================================
# 6. LIGHTGBM STREAMING DATASET
# ======================================================

# train_data = lgb.Dataset(
#     "train_clean.csv",
#     label="train_label.csv",
#     params={
#         "header": True,
#         "label_column": "isFraud",
#     }
# )

In [13]:
# print(train_transaction["isFraud"].dtype)
# print(train_transaction.dtypes)

Int64
[Float64, Int64, Float64, Float64, Int64, Float64, Float64, Float64, Int64, Float64, Int64, Float64, Float64, Float64, Float64, Int64, Int64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float6

In [10]:
# =======================================================
# 5. CONVERT TO LIGHTGBM DATASET ONCE (ANTI-CRASH)
# =======================================================
X = train_transaction.drop(target).to_pandas()  # safe (done once)
y = train_transaction[target].to_pandas()

train_lgb = lgb.Dataset(X, label=y)

In [ ]:
# ======================================================
# 7. OPTUNA TUNING (STREAMING MODE)
# ======================================================
# def objective(trial):
#     params = {
#         "objective": "binary",
#         "metric": "auc",
#         "boosting_type": "gbdt",
#         "verbosity": -1,
#         "num_leaves": trial.suggest_int("num_leaves", 31, 255),
#         "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05),
#         "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
#         "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
#         "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
#         "min_child_samples": trial.suggest_int("min_child_samples", 20, 200),
#         "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
#         "device": "cpu"
#     }

#     model = lgb.train(params, train_data)

#     # Evaluate on a small validation set (to keep RAM low)
#     val_sample = train_transaction.sample(50000)
#     X_val = val_sample.select(feature_cols).to_pandas()
#     y_val = val_sample[target].to_pandas()

#     preds = model.predict(X_val)
#     auc = roc_auc_score(y_val, preds)

#     return auc


# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10)  # keep small for safety

# best_params = study.best_params
# best_params.update({
#     "objective": "binary",
#     "metric": "auc",
#     "device": "cpu"
# })

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
params = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
}

tuner = lgb_tuner.LightGBMTunerCV(
    params,
    train_lgb,
    folds=folds,
    # fit_kwargs={
    #     "early_stopping_rounds": 50,
    #     "verbose_eval": False  # <-- pindahkan ke sini
    # }
)

tuner.run()  # <<== OPTUNA tidak crash karena LGBMTuner sangat stabil

best_params = tuner.best_params
best_score = tuner.best_score

print("Best params:", best_params)
print("Best AUC:", best_score)

[I 2025-12-02 14:30:12,034] A new study created in memory with name: no-name-ebadb0b4-fe64-4eba-bf71-b78b8576a864
feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:877: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(
feature_fraction, val_score: 0.962358:  14%|#4        | 1/7 [16:52<1:41:16, 1012.76s/it]/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:877: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(
feature_fraction, val_score: 0.962358:  29%|##8       | 2/7 [37:28<1:35:18, 1143.77s/it]/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:877: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(
feature_fraction, val_score: 0.962358:  43%|####2     | 3/7 [1:09:29<1:39:55, 1498.92s/it]/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_spl

In [ ]:
# ======================================================
# 8. TRAIN FINAL MODEL (STREAMING MODE)
# ======================================================
final_model = lgb.train(best_params, train_lgb)

In [ ]:
X_test = test_transaction.drop("TransactionID").to_pandas()
test_pred = final_model.predict(X_test)

submission = pl.DataFrame({
    "TransactionID": test_transaction["TransactionID"],
    "isFraud": test_pred
})

submission.write_csv("submission_done.csv")
print("Done! Saved as submission_done.csv")
